In [ ]:
import sys
import os
import math
import random
import bisect
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import subprocess
import re
import tempfile
import itertools
import torch
import spacy
import amrlib
import penman
import openai

from typing import List, Tuple
from operator import itemgetter 
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
# import qgrid

BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')
OPENAI_API_KEY=''
openai.api_key = OPENAI_API_KEY

from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.vlmapi import VLM_API
from nebula_api.atomic2020.comet_enrichment_api import *
from nebula_api.canonisation_api import CANON_API
import nebula_api.playground_api as pg_api

In [ ]:
nre = NRE_API()
api = RemoteAPIUtility()
vlm = VLM_API()
ascore = CANON_API()
# comet = Comet("/app/NEBULA2/nebula_api/atomic2020/comet-atomic_2020_BART")
# stog = amrlib.load_stog_model(model_dir="/app/NEBULA2/models/model_stog")
# gtos = amrlib.load_gtos_model(model_dir="/app/NEBULA2/models/model_gtos")
# model_name = "Alireza1044/albert-base-v2-cola" 


# Download cola model
# cola_model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def flatten(lst): return [x for l in lst for x in l]

def compute_batch_scores(video_emb: torch.Tensor, texts: List[str], normalize=True, **kwargs) -> List[float]:    
    emb_batch = vlm.encode_text(texts, **kwargs)
    if type(emb_batch) == list:
        emb_batch = torch.stack(emb_batch,axis=0)
    if normalize:
        video_emb = (video_emb / video_emb.norm(2)).squeeze()           # This should be done in vlm
        # print("normalized video norm: {}".format(video_emb.norm(2)))
        n = (emb_batch * emb_batch).sum(axis=1).sqrt()
        emb_batch = emb_batch / n.unsqueeze(1).expand_as(emb_batch)
        # print("normalized text norms:")
        # for emb in emb_batch:
        #     print(emb.norm(2))                        
    return (video_emb.unsqueeze(0).expand_as(emb_batch)*emb_batch).sum(dim=1).cpu().numpy()


def compute_concat_score(image_emb: torch.Tensor, texts: List[str], join_on=',') -> float:
    combined_text = ""
    for t in [x.strip() for x in texts]:
        if t[-1]=='.':
            t = t[:-1]       
        t+=join_on
        t+=' '
        combined_text+=t
    print("Combined: "+combined_text)
    return torch.matmul(image_emb,mdmmt.encode_text(combined_text.strip()) )       

In [ ]:
actionplace_engine = "text-curie-001"
actionplace_prompt_template = """Combine base, action, and place into one sentence.

Base: A man sitting on a bench in front of a building
Action: sit and think to himself
Place: at a train station in india
Combined: A man sits and thinks to himself at a train station in India

Base: A woman in a dress is standing in a field
Action: make her way to an empty field
Place: beside a meadow
Combined: A woman in a dress is walking to an empty field, besides a meadow

Base: A boat is in the water with a red flag
Action: chase the other boat
Place: aboard a navy ship
Combined: A small navy boat with a red flag speeds and chases another boat

Base: A dark image of a creature with a dark background
Action: shatter the reflection
Place: in a ruined courtyard
Combined: A dark image of a creature with a dark background is shattered in a ruined courtyard

Base: A woman standing in a doorway in a living room
Action: leave for home on her own
Place: just inside someone's front door of their house in the living room
Combined: A woman standing just inside the front door of someone's house is leaving home on her own

Base: A man in a suit and tie standing in front of a wall
Action: Take out a passport
Place: outside of a terminal baggage claim
Combined: A man in a suit and a tie takes out his passport outside of a terminal's baggage claim

Base: A woman pushing a suitcase with a man standing behind her
Action: set the suitcase by the door
Place: in a 19th century house
Combined: A woman sets her suitcase by the door in a 19th century house, with a man standing behind her

Base: A woman walking down a street in the dark
Action: try to evade a car
Place: on a narrow street or alley
Combined: A woman in a narrow street or alley tries to evade a car

Base: A woman walking down a street in the dark
Action: trip while she is trying to hurry
Place: on a narrow street or alley
Combined: A woman trips while she is trying to hurry in a narrow street or alley

Base: A car is driving down a dark street at night
Action: drive away quickly from the scene
Place: outside on a street at night that is filled with police cars
Combined:  A car outside on a street filled with police cars is quickly driving away from the scene

Base: A woman with a red tie is driving a car
Action: hang on tightly to the dashboard
Place: in a police car
Combined: A woman with a red tie in a police car hangs on tightly to the dashboard

Base: A man standing in front of a doorway at night
Action: turns on the light
Place: just outside of someone's home off of the porch
Combined: A man turns on the light just outside of someone's home off of the porch

Base: A man standing in front of a doorway at night
Action: call out to see if anyone is home
Place: just outside of someone's home off of the porch
Combined: A man calls out to see if anyone is home, just outside of someone's home off of the porch

Base: A young man practices karate
Action: leaps over the hedge to the next yard
Place: outside someone's home on a porch
Combined: A young man  leaps over the hedge to the next yard outside someone's home on a porch

Base: Two men standing next to each other in front of a forest
Action: stare off into the night sky
Place: near a cornfield outside someone's house
Combined: Two man stare off into the night sky near a cornfield outside someone's house

Base: A man with sunglasses and a yellow jacket standing next to a man with a yellow jacket
Action: scan the horizon
Place: at a gas station in the desert
Combined: At a gas station in the desert, a man with sunglasses and a yellow jacket scans the horizon

Base: A man in a suit and tie holding a knife
Action: present the notes to his superiors
Place: government capitol
Combined: A man in a suit and tie presents the notes to his superiors in the government capitol

Base: {}
Action: {}
Place: {}
Combined:"""


def gpt_execute(prompt_template, *args, **kwargs):    
    prompt = prompt_template.format(*args)
    response = openai.Completion.create(prompt=prompt,temperature=0.6, max_tokens=128, **kwargs)
    return response

In [ ]:
mid, elem = ('Movies/114206337', 1)
emb_video = vlm.encode_video(mid,elem)